In [1]:
import sys
sys.path.append('../scripts/')
from read_untils import read_file, create_folder, temp_record_query, temp_record_sdf

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = (
    # Create a spark session (which will run spark jobs)
    SparkSession.builder.appName("Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config('spark.executor.memory','10g')
    .config('spark.driver.memory','12g')
    .config('spark.driver.maxResultsSize', '10 GiB')
    .config('spark.shuffle.file.buffer', '64k')
    # .config("spark.network.timeout", "3600s")
    # .master("local[6]")
    .getOrCreate()
    )

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/10 01:31:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/10 01:31:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/10 01:31:49 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/10/10 01:31:49 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [4]:
import pandas as pd

### cases_daily_aus_NET

In [5]:
print("================ Cases ================")
ncd = read_file(spark, 'external_cases_daily_aus_NET.csv', type='csv')

================ Cases ================
|> Loading File...
|> Loading Finished!
-RECORD 0-------------------
 Date            | 25/01/20 
 New cases / day | 4        
only showing top 1 row



In [6]:
temp = ncd.dropna(how='any').count()
print( f'There are {ncd.count()} data, and {temp} have no null value.')

There are 980 data, and 980 have no null value.


In [7]:
from pyspark.sql import functions as F
# from pyspark.sql.functions import to_date, to_timestamp
ncd_new = ncd.withColumn(
    'Date',
    F.to_date('Date', 'dd/MM/yy')
)
ncd_new.printSchema()

root
 |-- Date: date (nullable = true)
 |-- New cases / day: string (nullable = true)



In [8]:
temp_record_sdf(ncd_new, '../data/curated/external_ncd.parquet', overwrite=True)

|> Waitting for saving...


|> Save Successfully!


### retail_trade

In [9]:
data = pd.read_csv('../data/tables/external_retail_trade.csv')
head = data[:9]
df = data[9:]

In [10]:
head

,Unnamed: 0,Unnamed: 0.1,Turnover ; Total (State) ; Food retailing ;,Turnover ; Total (State) ; Household goods retailing ;,"Turnover ; Total (State) ; Clothing, footwear and personal accessory retailing ;",Turnover ; Total (State) ; Department stores ;,Turnover ; Total (State) ; Other retailing ;,"Turnover ; Total (State) ; Cafes, restaurants and takeaway food services ;",Turnover ; Total (State) ; Total (Industry) ;,Turnover ; Total (State) ; Food retailing ;.1,...,Turnover ; Total (State) ; Other retailing ;.1,"Turnover ; Total (State) ; Cafes, restaurants and takeaway food services ;.1",Turnover ; Total (State) ; Total (Industry) ;.1,Turnover ; Total (State) ; Food retailing ;.2,Turnover ; Total (State) ; Household goods retailing ;.2,"Turnover ; Total (State) ; Clothing, footwear and personal accessory retailing ;.2",Turnover ; Total (State) ; Department stores ;.2,Turnover ; Total (State) ; Other retailing ;.2,"Turnover ; Total (State) ; Cafes, restaurants and takeaway food services ;.2",Turnover ; Total (State) ; Total (Industry) ;.2
0,0,Unit,$ Millions,$ Millions,$ Millions,$ Millions,$ Millions,$ Millions,$ Millions,$ Millions,...,$ Millions,$ Millions,$ Millions,$ Millions,$ Millions,$ Millions,$ Millions,$ Millions,$ Millions,$ Millions
1,1,Series Type,Original,Original,Original,Original,Original,Original,Original,Seasonally Adjusted,...,Seasonally Adjusted,Seasonally Adjusted,Seasonally Adjusted,Trend,Trend,Trend,Trend,Trend,Trend,Trend
2,2,Data Type,FLOW,FLOW,FLOW,FLOW,FLOW,FLOW,FLOW,FLOW,...,FLOW,FLOW,FLOW,FLOW,FLOW,FLOW,FLOW,FLOW,FLOW,FLOW
3,3,Frequency,Month,Month,Month,Month,Month,Month,Month,Month,...,Month,Month,Month,Month,Month,Month,Month,Month,Month,Month
4,4,Collection Month,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
5,5,Series Start,1982-04-01 00:00:00,1982-04-01 00:00:00,1982-04-01 00:00:00,1982-04-01 00:00:00,1982-04-01 00:00:00,1982-04-01 00:00:00,1982-04-01 00:00:00,1982-04-01 00:00:00,...,1982-04-01 00:00:00,1982-04-01 00:00:00,1982-04-01 00:00:00,1982-04-01 00:00:00,1982-04-01 00:00:00,1982-04-01 00:00:00,1982-04-01 00:00:00,1982-04-01 00:00:00,1982-04-01 00:00:00,1982-04-01 00:00:00
6,6,Series End,2022-07-01 00:00:00,2022-07-01 00:00:00,2022-07-01 00:00:00,2022-07-01 00:00:00,2022-07-01 00:00:00,2022-07-01 00:00:00,2022-07-01 00:00:00,2022-07-01 00:00:00,...,2022-07-01 00:00:00,2022-07-01 00:00:00,2022-07-01 00:00:00,2022-07-01 00:00:00,2022-07-01 00:00:00,2022-07-01 00:00:00,2022-07-01 00:00:00,2022-07-01 00:00:00,2022-07-01 00:00:00,2022-07-01 00:00:00
7,7,No. Obs,484,484,484,484,484,484,484,484,...,484,484,484,484,484,484,484,484,484,484
8,8,Series ID,A3348591K,A3348600A,A3348609W,A3348618X,A3348627A,A3348636C,A3348582J,A3348594T,...,A3348630R,A3348639K,A3348585R,A3348597X,A3348606R,A3348615T,A3348624V,A3348633W,A3348642X,A3348588W


In [11]:
df = df.reset_index()
df = df[list(df.columns)[2:]]
df

,Unnamed: 0.1,Turnover ; Total (State) ; Food retailing ;,Turnover ; Total (State) ; Household goods retailing ;,"Turnover ; Total (State) ; Clothing, footwear and personal accessory retailing ;",Turnover ; Total (State) ; Department stores ;,Turnover ; Total (State) ; Other retailing ;,"Turnover ; Total (State) ; Cafes, restaurants and takeaway food services ;",Turnover ; Total (State) ; Total (Industry) ;,Turnover ; Total (State) ; Food retailing ;.1,Turnover ; Total (State) ; Household goods retailing ;.1,...,Turnover ; Total (State) ; Other retailing ;.1,"Turnover ; Total (State) ; Cafes, restaurants and takeaway food services ;.1",Turnover ; Total (State) ; Total (Industry) ;.1,Turnover ; Total (State) ; Food retailing ;.2,Turnover ; Total (State) ; Household goods retailing ;.2,"Turnover ; Total (State) ; Clothing, footwear and personal accessory retailing ;.2",Turnover ; Total (State) ; Department stores ;.2,Turnover ; Total (State) ; Other retailing ;.2,"Turnover ; Total (State) ; Cafes, restaurants and takeaway food services ;.2",Turnover ; Total (State) ; Total (Industry) ;.2
0,1982-04-01 00:00:00,1162.6,592.3,359.9,460.1,479.1,342.4,3396.4,1167.1,653.5,...,507.9,349.8,3518.7,1173.2,652.9,362.7,482.4,505.1,347.5,3523.4
1,1982-05-01 00:00:00,1150.9,629.6,386.6,502.6,486.1,342.1,3497.9,1178.5,648.6,...,502.3,346.4,3527.7,1181.6,654.2,362,484.2,504.9,346.3,3533.6
2,1982-06-01 00:00:00,1160,607.4,350.5,443.8,467.5,328.7,3357.8,1203.6,655.9,...,506.9,350.8,3561.5,1192.7,655.7,361.9,486.5,504.7,345.8,3547
3,1982-07-01 00:00:00,1206.4,632.4,359.3,459.1,491.1,338.5,3486.8,1209.7,660.5,...,503.6,341.5,3553.9,1203.1,656.7,361.6,488.6,505.4,345.8,3560.6
4,1982-08-01 00:00:00,1152.5,622.6,325.2,438.4,485.7,331.5,3355.9,1198.8,659.7,...,506,342.7,3581.8,1213.3,656.6,361.9,489.8,505.6,346.4,3573.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
479,2022-03-01 00:00:00,13052.6,5875.3,2608.7,1599.3,5103.4,4554.2,32793.5,12939.6,6188.2,...,5290.5,4555.5,33626.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
480,2022-04-01 00:00:00,13033.5,5463.1,2767.8,1725,4898.5,4610.4,32498.3,13182.7,6023.3,...,5317.8,4704,33923.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
481,2022-05-01 00:00:00,12903.6,5769,2873.4,1767.6,5195.9,4748.9,33258.5,13196.8,6046.6,...,5396.8,4787.1,34172,NaN,NaN,NaN,NaN,NaN,NaN,NaN
482,2022-06-01 00:00:00,12469.6,6010.8,2868.6,1817.9,5166.2,4672.5,33005.5,13160.9,6028.6,...,5422.7,4914.6,34239.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df=df.rename({list(df.columns)[0]: 'month'}, axis=1)

In [13]:
df['month'] = df['month'].astype('datetime64[ns]').dt.date

In [14]:
df = df[df.month >= pd.to_datetime('2021-02')]
df = df.reset_index()
df = df[list(df.columns)[1:]]
df

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


,month,Turnover ; Total (State) ; Food retailing ;,Turnover ; Total (State) ; Household goods retailing ;,"Turnover ; Total (State) ; Clothing, footwear and personal accessory retailing ;",Turnover ; Total (State) ; Department stores ;,Turnover ; Total (State) ; Other retailing ;,"Turnover ; Total (State) ; Cafes, restaurants and takeaway food services ;",Turnover ; Total (State) ; Total (Industry) ;,Turnover ; Total (State) ; Food retailing ;.1,Turnover ; Total (State) ; Household goods retailing ;.1,...,Turnover ; Total (State) ; Other retailing ;.1,"Turnover ; Total (State) ; Cafes, restaurants and takeaway food services ;.1",Turnover ; Total (State) ; Total (Industry) ;.1,Turnover ; Total (State) ; Food retailing ;.2,Turnover ; Total (State) ; Household goods retailing ;.2,"Turnover ; Total (State) ; Clothing, footwear and personal accessory retailing ;.2",Turnover ; Total (State) ; Department stores ;.2,Turnover ; Total (State) ; Other retailing ;.2,"Turnover ; Total (State) ; Cafes, restaurants and takeaway food services ;.2",Turnover ; Total (State) ; Total (Industry) ;.2
0,2021-02-01,11359.1,4879.3,1872.8,1197.7,4008.5,3440.4,26757.8,12350.5,5535.2,...,4534.8,3853.2,30332.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-03-01,12539.7,5337.3,2297.3,1600.2,4491.2,4044.9,30310.7,12297,5544.4,...,4584.1,4025.6,30725.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-04-01,11981.3,5011.2,2387.6,1596.8,4252.1,4017.5,29246.6,12373.2,5605.9,...,4648.8,4101.7,30960,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-05-01,12310.8,5249.2,2519.6,1602.1,4468,4043.1,30192.8,12487.4,5551.9,...,4670.7,4119.5,31007.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-06-01,11922.3,5551.9,2265.2,1560.2,4410.4,3696.9,29406.8,12655.1,5514.1,...,4642.1,3891,30563.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2021-07-01,12788.2,5199.7,1841.5,1313.9,4558.3,3447.7,29149.3,12910.9,5392.5,...,4680.2,3427.4,29761.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2021-08-01,12977.1,5057.1,1471.6,1032.5,4593.1,3219.9,28351.3,13224.5,5257.4,...,4716.3,3190.8,29261.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2021-09-01,12743.8,5330.7,1616.5,1137.6,4678.1,3369.8,28876.5,13075.4,5487.7,...,4825.1,3363.3,29756.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2021-10-01,13224,5907.5,2173.8,1493.4,4959.2,3916.1,31674.1,12981.5,5702.2,...,4910.7,3776.1,31132.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2021-11-01,12696,7018.2,3241,2106.2,5860.2,4222.7,35144.3,12683.6,6401.3,...,5246.7,4105.2,33345,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df.count()

month                                                                                   18
Turnover ;  Total (State) ;  Food retailing ;                                           18
Turnover ;  Total (State) ;  Household goods retailing ;                                18
Turnover ;  Total (State) ;  Clothing, footwear and personal accessory retailing ;      18
Turnover ;  Total (State) ;  Department stores ;                                        18
Turnover ;  Total (State) ;  Other retailing ;                                          18
Turnover ;  Total (State) ;  Cafes, restaurants and takeaway food services ;            18
Turnover ;  Total (State) ;  Total (Industry) ;                                         18
Turnover ;  Total (State) ;  Food retailing ;.1                                         18
Turnover ;  Total (State) ;  Household goods retailing ;.1                              18
Turnover ;  Total (State) ;  Clothing, footwear and personal accessory retailing ;.1    18

In [16]:
df = df[list(df.columns)[:-7]]
df.head()

,month,Turnover ; Total (State) ; Food retailing ;,Turnover ; Total (State) ; Household goods retailing ;,"Turnover ; Total (State) ; Clothing, footwear and personal accessory retailing ;",Turnover ; Total (State) ; Department stores ;,Turnover ; Total (State) ; Other retailing ;,"Turnover ; Total (State) ; Cafes, restaurants and takeaway food services ;",Turnover ; Total (State) ; Total (Industry) ;,Turnover ; Total (State) ; Food retailing ;.1,Turnover ; Total (State) ; Household goods retailing ;.1,"Turnover ; Total (State) ; Clothing, footwear and personal accessory retailing ;.1",Turnover ; Total (State) ; Department stores ;.1,Turnover ; Total (State) ; Other retailing ;.1,"Turnover ; Total (State) ; Cafes, restaurants and takeaway food services ;.1",Turnover ; Total (State) ; Total (Industry) ;.1
0,2021-02-01,11359.1,4879.3,1872.8,1197.7,4008.5,3440.4,26757.8,12350.5,5535.2,2385.8,1673.1,4534.8,3853.2,30332.5
1,2021-03-01,12539.7,5337.3,2297.3,1600.2,4491.2,4044.9,30310.7,12297,5544.4,2470.8,1804,4584.1,4025.6,30725.9
2,2021-04-01,11981.3,5011.2,2387.6,1596.8,4252.1,4017.5,29246.6,12373.2,5605.9,2539.2,1691.2,4648.8,4101.7,30960
3,2021-05-01,12310.8,5249.2,2519.6,1602.1,4468,4043.1,30192.8,12487.4,5551.9,2506.3,1671.9,4670.7,4119.5,31007.7
4,2021-06-01,11922.3,5551.9,2265.2,1560.2,4410.4,3696.9,29406.8,12655.1,5514.1,2297.7,1563.9,4642.1,3891,30563.8


In [17]:
df_origin = df[list(df.columns)[:8]]
df_origin.head()

,month,Turnover ; Total (State) ; Food retailing ;,Turnover ; Total (State) ; Household goods retailing ;,"Turnover ; Total (State) ; Clothing, footwear and personal accessory retailing ;",Turnover ; Total (State) ; Department stores ;,Turnover ; Total (State) ; Other retailing ;,"Turnover ; Total (State) ; Cafes, restaurants and takeaway food services ;",Turnover ; Total (State) ; Total (Industry) ;
0,2021-02-01,11359.1,4879.3,1872.8,1197.7,4008.5,3440.4,26757.8
1,2021-03-01,12539.7,5337.3,2297.3,1600.2,4491.2,4044.9,30310.7
2,2021-04-01,11981.3,5011.2,2387.6,1596.8,4252.1,4017.5,29246.6
3,2021-05-01,12310.8,5249.2,2519.6,1602.1,4468,4043.1,30192.8
4,2021-06-01,11922.3,5551.9,2265.2,1560.2,4410.4,3696.9,29406.8


In [18]:
df_seasonal = df[ list(df.columns)[:1] + list(df.columns)[8:]]
df_seasonal.head()

,month,Turnover ; Total (State) ; Food retailing ;.1,Turnover ; Total (State) ; Household goods retailing ;.1,"Turnover ; Total (State) ; Clothing, footwear and personal accessory retailing ;.1",Turnover ; Total (State) ; Department stores ;.1,Turnover ; Total (State) ; Other retailing ;.1,"Turnover ; Total (State) ; Cafes, restaurants and takeaway food services ;.1",Turnover ; Total (State) ; Total (Industry) ;.1
0,2021-02-01,12350.5,5535.2,2385.8,1673.1,4534.8,3853.2,30332.5
1,2021-03-01,12297,5544.4,2470.8,1804,4584.1,4025.6,30725.9
2,2021-04-01,12373.2,5605.9,2539.2,1691.2,4648.8,4101.7,30960
3,2021-05-01,12487.4,5551.9,2506.3,1671.9,4670.7,4119.5,31007.7
4,2021-06-01,12655.1,5514.1,2297.7,1563.9,4642.1,3891,30563.8


In [20]:
cols = ['month', 'food', 'household goods', 'clothing footwear and personal accessory', 'department store', 'other', 'cafes restaurants and takeway foods', 'total']
sdf_origin = spark.createDataFrame(df_origin, cols)
sdf_seasonal = spark.createDataFrame(df_seasonal, cols)

In [21]:
sdf_seasonal = sdf_seasonal.withColumn('month', sdf_seasonal['month'].cast('string'))

In [22]:
mean = sdf_seasonal.agg(*(F.mean(c).alias(c) for c in sdf_seasonal.columns[1:]))
meaninfo = mean.first().asDict()
meaninfo

{'food': 12882.772222222222,
 'household goods': 5770.094444444444,
 'clothing footwear and personal accessory': 2512.4444444444443,
 'department store': 1651.1111111111113,
 'other': 4993.483333333334,
 'cafes restaurants and takeway foods': 4135.805555555556,
 'total': 31945.705555555553}

In [25]:
data = [('2020-08-01', '12882.77', '5770.09', '2512.44', '1651.11', '4993.48', '4135.81', '31945.71')]
august_data = spark.createDataFrame(data, sdf_seasonal.columns)
sdf_seasonal = sdf_seasonal.union(august_data)

In [26]:
temp_record_sdf(sdf_origin, '../data/curated/external_retail_trade_origin.parquet', overwrite=True)
temp_record_sdf(sdf_seasonal, '../data/curated/external_retail_trade_seasonal.parquet', overwrite=True)

|> Waitting for saving...


|> Save Successfully!
|> Waitting for saving...


|> Save Successfully!
